## Airborne Tool
Author: Harrison Baker


In [3]:
# package imports
import pandas as pd
import re
import numpy as np
from functools import reduce


Setting up adjustment information

In [4]:
# important variables defined

periods = ["MS", "Day", "OOH", "Eve", "ES", "Night", "SS", "SA"]

minValue = 25

highNoiseAffected = 75

countDict = {
    "MS": [0, 5, 15, 25],
    "Day": [0, 10, 20, 30],
    "OOH": [0, 5, 15, 25],
    "Eve": [0, 5, 15, 25],
    "ES": [0, 5, 15, 25],
    "Night": [0, 5, 15, 25],
    "SS": [0],
    "SA": [0]
}

scenarioSettings = ["Scenario Name", "Scenario Description", "Variant Number"]
scenarioSettings.extend(periods)


### Adjustment for Scenario DataFrame

For each new scenario add a new row (follow the exiting patern). Print the existing DF if you are unsure. Numerical value for scenario represents the Lw of the activity assuming that it was modelled in CadnaA as Lw = 100. If modelling was done with real Lw values, set all 'adjustment' values to 100 so there is no change to the Lw. Format is:
| Scenario Name | Scenario Description | Variant Number | MS Adj | Day Adj | Day OOH Adj | Eve Adj | ES Adj | Night Adj | SS Adj | SA Adj |
| :------- | :------: | :------: | :------: | :------: | :------: | :------: | :------: | :------: | :------: | -------: |
| ABC | Rockhammering shaft | 1 | 100 | 100 | 100 | 100 | 100 | 100 | 100 | 100 |

In [5]:
# adjustment dictionary

adjustmentDf = pd.DataFrame(columns=scenarioSettings)
adjustmentDf.loc[0] = ["SCEN1", "a brief description", 1, np.nan, 150, 150, np.nan,  90, 110, 120, 80]
adjustmentDf.loc[1] = ["SCEN2", "a brief description", 5, np.nan, 150, 150, np.nan,  90, 110, 120, 80]
adjustmentDf.loc[2] = ["SCEN3", "a brief description", 4, np.nan, 150, 150, np.nan,  90, 110, 120, 80]

print(adjustmentDf)

  Scenario Name Scenario Description  Variant Number  MS  Day  OOH  Eve  ES  \
0         SCEN1  a brief description               1 NaN  150  150  NaN  90   
1         SCEN2  a brief description               5 NaN  150  150  NaN  90   
2         SCEN3  a brief description               4 NaN  150  150  NaN  90   

   Night   SS  SA  
0    110  120  80  
1    110  120  80  
2    110  120  80  


In [6]:
minAdjustment = pd.DataFrame(columns=scenarioSettings)
minAdjustment.loc[0] = ["SCEN1", "a brief description", 1, np.nan, 100, 100, np.nan,  80, 100, 100, 70]
minAdjustment.loc[1] = ["SCEN2", "a brief description", 5, np.nan, 100, 100, np.nan,  80, 100, 105, 70]
minAdjustment.loc[2] = ["SCEN3", "a brief description", 4, np.nan, 100, 100, np.nan,  70, 100, 110, 70]

print(minAdjustment)

  Scenario Name Scenario Description  Variant Number  MS  Day  OOH  Eve  ES  \
0         SCEN1  a brief description               1 NaN  100  100  NaN  80   
1         SCEN2  a brief description               5 NaN  100  100  NaN  80   
2         SCEN3  a brief description               4 NaN  100  100  NaN  70   

   Night   SS  SA  
0    100  100  70  
1    100  105  70  
2    100  110  70  


### Count Bin Dictionary

In [7]:
countDict = {
    "MS": [0, 5, 15, 25],
    "Day": [0, 10, 20, 30],
    "OOH": [0, 5, 15, 25],
    "Eve": [0, 5, 15, 25],
    "ES": [0, 5, 15, 25],
    "Night": [0, 5, 15, 25],
    "SS": [0],
    "SA": [0]
}

open cadnaA excel file, for each unique NCA_ID take the max value for each variant, adjust for scenario and write results df grouped by period

In [8]:
# open cadnaA results as df

cadnaA = pd.read_excel("RawCadnaA.xlsx")
dataColumns = cadnaA.columns[cadnaA.columns.get_loc(1):] # doesn't include NMLs
infoColumns = cadnaA.columns[:cadnaA.columns.get_loc("Day_NML")] # info up to NMLs

def reduceToMax(df: pd.DataFrame, nca_id = "NCA_ID", ncaTool = "NCA_tool"):
    max_df = df.groupby(nca_id, as_index = False).max().sort_values(by=ncaTool).set_index(nca_id, drop = False)
    max_df.loc[max_df[ncaTool] == "OSR", ncaTool] = max_df[nca_id].str.split('-').str[0] # note that MXU are assumed to have OSR tool with NCAXX
    return max_df

reduced = reduceToMax(cadnaA)

def calcPeriodLevels(row, corrections: pd.Series, scenarioName, variantId):
    # shout out AMo for this function
    if not variantId in row.index:
        return row
    for periodName, periodValue in corrections.items():
        if periodValue == np.nan:
            continue
        newLevel = row[variantId] + periodValue - 100
        row[f"{scenarioName}_{periodName}"] = newLevel
    return row

def rmValsBelowFloor(x):
    if isinstance(x, int):
        if x < minValue:
            return "-"
        else:
            return x
    return x

def applyAdjustment(df: pd.DataFrame, periods, infoColumns, dataColumns, adjustmentDf, ncaTool = "NCA_tool"):
    
    base = df.copy()
    
    adjusted = base[infoColumns].copy()
    data = base[dataColumns].copy().round(0).astype(int)

    mask = adjusted[ncaTool].str.startswith("NCA")
    base["SS_NML"] = np.where(mask, np.maximum(base["Night_NML"] + 10, 52), 999) # discuss with MT
    base["SA_NML"] = np.where(mask, np.maximum(base["SS_NML"], 65), 999)
    base["OOH_NML"] = np.where(mask, base["Day_NML"] - 5, base["Day_NML"])

    scenarioDict= {
        x: None for x in adjustmentDf["Scenario Name"]
    }

    for id, s in adjustmentDf.iterrows():
        scenarioName = s["Scenario Name"]
        variantId = s["Variant Number"]
        corrections = s[periods]
        scenarioDf = data.copy().astype(int)
        scenarioDf = scenarioDf.apply(calcPeriodLevels, scenarioName = scenarioName, variantId = variantId, corrections = corrections, axis = 1)
        scenarioDf = scenarioDf.dropna(axis=1, how='all').astype(int)#.map(rmValsBelowFloor)
        # for col in scenarioDf.columns:
        #     scenarioDf[col] = scenarioDf[col].apply(lambda x: "-" if x < minValue else x)
        scenarioDict[scenarioName] = scenarioDf


    for p in periods: # figure out how to remove nested for loop 
        for name, subDf in scenarioDict.items():
            cols = [c for c in subDf.columns if isinstance(c, str) and '_' in c and c.split('_')[-1] == p]
            if not cols:
                continue

            adjusted[f"{p}_NML"] = base[f"{p}_NML"]

            adjusted = adjusted.join(subDf[cols])

    adjusted.sort_values(by=[ncaTool, "Address"], inplace = True) # note that if inplace isnt set as True it will not update the existing dataframe
    
    return adjusted, data, scenarioDict

adjusted, data, scenarioDict = applyAdjustment(reduced, periods, infoColumns, dataColumns, adjustmentDf)

# adjusted.to_csv('testFullTable.csv', index=False)

print(adjusted.head())

                          NCA_tool                     NCA_ID  \
NCA_ID                                                          
RES-NCA06-00190002497        NCA06      RES-NCA06-00190002497   
OSR_MXU-NCA06-00190007454    NCA06  OSR_MXU-NCA06-00190007454   
OSR_MXU-NCA07-00190007465    NCA07  OSR_MXU-NCA07-00190007465   
OSR_MXU-NCA07-00190007424    NCA07  OSR_MXU-NCA07-00190007424   
OSR_COM-NCA07-00190002328  OSR_COM  OSR_COM-NCA07-00190002328   

                                                           Address  Num_Units  \
NCA_ID                                                                          
RES-NCA06-00190002497            168-170 KENT STREET MILLERS POINT          1   
OSR_MXU-NCA06-00190007454  200  CUMBERLAND STREET, THE ROCKS (SYDN          1   
OSR_MXU-NCA07-00190007465         18A PITT STREET, SYDNEY (SYDNEY)          1   
OSR_MXU-NCA07-00190007424       38  BRIDGE STREET, SYDNEY (SYDNEY)          1   
OSR_COM-NCA07-00190002328                    1 BLIGH STREE

### make the range dataframe if wanted

In [12]:
filteredColumns = [name for name in adjusted.columns if adjustmentDf["Scenario Name"].str.contains(name.split('_')[0]).any()]

lowerLevelsDf, _, _ = applyAdjustment(reduced, periods, infoColumns, dataColumns, minAdjustment)

def makeLevelsDf(lowerDf, upperDf, filteredColumns, minValue, nca_id = "NCA_ID"):
    
    levelsRangeDf = upperDf.copy()

    for col in filteredColumns:
        levelsRangeDf[col] = np.where(upperDf[col] < minValue, "-", np.where(
            ((lowerDf[col] < minValue) & (upperDf[col] >= minValue)), "<" + upperDf[col].astype(str), 
            lowerDf[col].astype(str) + " - " + upperDf[col].astype(str)))
        
    return levelsRangeDf

levelsRangeDf = makeLevelsDf(lowerLevelsDf, adjusted, filteredColumns, minValue)

print(levelsRangeDf)




                          NCA_tool                     NCA_ID  \
NCA_ID                                                          
RES-NCA06-00190002497        NCA06      RES-NCA06-00190002497   
OSR_MXU-NCA06-00190007454    NCA06  OSR_MXU-NCA06-00190007454   
OSR_MXU-NCA07-00190007465    NCA07  OSR_MXU-NCA07-00190007465   
OSR_MXU-NCA07-00190007424    NCA07  OSR_MXU-NCA07-00190007424   
OSR_COM-NCA07-00190002328  OSR_COM  OSR_COM-NCA07-00190002328   
OSR_COM-NCA07-00190002373  OSR_COM  OSR_COM-NCA07-00190002373   
OSR_COM-NCA07-00190002372  OSR_COM  OSR_COM-NCA07-00190002372   
OSR_COM-NCA07-00190002346  OSR_COM  OSR_COM-NCA07-00190002346   
OSR_COM-NCA06-00190007403  OSR_COM  OSR_COM-NCA06-00190007403   
OSR_COM-NCA06-00190007395  OSR_COM  OSR_COM-NCA06-00190007395   
OSR_COM-NCA06-00190007393  OSR_COM  OSR_COM-NCA06-00190007393   
OSR_COM-NCA07-S372000018   OSR_COM   OSR_COM-NCA07-S372000018   
OSR_COM-NCA07-00190002349  OSR_COM  OSR_COM-NCA07-00190002349   
OSR_COM-NCA07-00280002222

In [ ]:
# filteredColumns = [name for name in adjusted.columns if adjustmentDf["Scenario Name"].str.contains(name.split('_')[0]).any()]

# def countExceedances(binsDictionary, resultsDf, resultsColumns):
#     countDictionary = {}
#     for name in resultsColumns:
#        period = name.split('_')[1]
#        bins = binsDictionary[period] + [np.inf]
#        bins = [x + 0.1 for x in bins] # np.histogram treats bins as [), adding 0.1 'changes' this to (] for floats (given results are now floats)
#        periodNml = resultsDf[f"{period}_NML"]
#        colValues = resultsDf[name]
#        diffs = np.asarray(colValues - periodNml)
#        hist, _ = np.histogram(diffs, bins = bins) 
#        countDictionary[name] = hist.tolist()

#     return countDictionary

# countTest = countExceedances(countDict, adjusted, filteredColumns)



# print(countTest)

{'SCEN1_Day': [0, 4, 5, 11], 'SCEN2_Day': [2, 0, 4, 13], 'SCEN3_Day': [1, 0, 5, 11], 'SCEN1_OOH': [0, 1, 3, 16], 'SCEN2_OOH': [1, 2, 4, 13], 'SCEN3_OOH': [0, 1, 0, 16], 'SCEN1_ES': [0, 0, 0, 0], 'SCEN2_ES': [0, 0, 0, 0], 'SCEN3_ES': [0, 0, 0, 0], 'SCEN1_Night': [1, 2, 1, 0], 'SCEN2_Night': [5, 3, 2, 0], 'SCEN3_Night': [4, 1, 2, 0], 'SCEN1_SS': [0], 'SCEN2_SS': [0], 'SCEN3_SS': [0], 'SCEN1_SA': [0], 'SCEN2_SA': [0], 'SCEN3_SA': [0]}


In [13]:
def countExceedances(binsDictionary, resultsDf, resultsColumns, recStr, highNoiseAffected=highNoiseAffected):
    countDictionary = {}
    for name in resultsColumns:
       period = name.split('_')[1]
       bins = binsDictionary[period] + [np.inf]
       bins = [x + 0.1 for x in bins] # np.histogram treats bins as [), adding 0.1 'changes' this to (] for floats (given results are now floats)
       periodNml = resultsDf[f"{period}_NML"]
       colValues = resultsDf[name]
       diffs = np.asarray(colValues - periodNml)
       hist, edges = np.histogram(diffs, bins = bins)
       subDict = {
           (
               f"{(round(edges[i], 0)).astype(int)} to {(round(edges[i+1], 0)).astype(int)}"
                  if edges[i+1] != np.inf
                  else f"> {(round(edges[i], 0)).astype(int)}"
           ): hist[i]
           for i in range(len(hist))}
       if period == "Day" and recStr.startswith("NCA"):
            subDict[f"{highNoiseAffected}dBA or greater"] = (colValues > highNoiseAffected).sum()
       subDictInt = {key: int(value) for key, value in subDict.items()}
       countDictionary[name] = subDictInt

    countDf = pd.DataFrame.from_dict(countDictionary, orient = 'index').stack(future_stack=False).astype(int)
    return countDf

countTest = countExceedances(countDict, adjusted, filteredColumns, recStr="NCA08")

print(countTest)

SCEN1_Day    0 to 10              0
             10 to 20             4
             20 to 30             5
             > 30                11
             75dBA or greater    19
SCEN2_Day    0 to 10              2
             10 to 20             0
             20 to 30             4
             > 30                13
             75dBA or greater    19
SCEN3_Day    0 to 10              1
             10 to 20             0
             20 to 30             5
             > 30                11
             75dBA or greater    16
SCEN1_OOH    0 to 5               0
             5 to 15              1
             15 to 25             3
             > 25                16
SCEN2_OOH    0 to 5               1
             5 to 15              2
             15 to 25             4
             > 25                13
SCEN3_OOH    0 to 5               0
             5 to 15              1
             15 to 25             0
             > 25                16
SCEN1_ES     0 to 5         

In [39]:
def countForType(resultsDf, countDict, filteredColumns, ncaTool = "NCA_tool"):
    recTypes = resultsDf[ncaTool].unique()
    countTypeDict = {}
    for recType in recTypes:
        recString=str(recType)
        typeDf = resultsDf[resultsDf[ncaTool] == recType]
        typeCountDf = countExceedances(countDict, typeDf, filteredColumns, recStr=recString)
        countTypeDict[recType] = typeCountDf
    
    countTypeDf = pd.DataFrame(countTypeDict)
    return countTypeDf

def makeWideCountTable(resultsDf, countDict, filteredColumns, ncaTool = "NCA_tool"):

    typeCount = countForType(adjusted, countDict, filteredColumns, ncaTool)

    stacked = typeCount.stack().reset_index().rename(
        columns = {
            "level_0": "Scenario",
            "level_1": "Bin",
            "level_2": "Receiver",
            0: "Count"
        }
    )

    stacked["Period"] = stacked["Scenario"].str.split('_').str[1]

    activePeriods=list(x for x in periods if x in stacked["Period"].values)

    pCounts ={}

    for p in activePeriods:
        df_p = stacked[stacked["Period"] == p]
        if df_p.empty:
            continue
        
        w = pd.pivot_table(
            df_p,
            values="Count",
            index=["Receiver", "Bin"],
            columns=["Scenario"],
            aggfunc="sum"
        ).reset_index()

        w = w.sort_values(["Receiver", "Bin"])

        w["row_id"] = w.groupby("Receiver").cumcount()

        w = w.rename(columns={'Bin': f'Bins_{p}'})
        
        pCounts[p] = w

    pCountsDfs = list(pCounts.values())

    wide = reduce(
        lambda left, right: pd.merge(left, right, on=["Receiver", "row_id"], how="outer"), pCountsDfs
    )

    wide = wide.sort_values(["Receiver", "row_id"]).set_index("Receiver").drop(columns=["row_id"]).fillna("")

    return wide, stacked 

wide, stacked = makeWideCountTable(adjusted, countDict, filteredColumns)

# wide.to_csv('testCountTable.csv', index=True)

print(wide)


Scenario          Bins_Day  SCEN1_Day  SCEN2_Day  SCEN3_Day  Bins_OOH  \
Receiver                                                                
NCA06              0 to 10        0.0        0.0        0.0    0 to 5   
NCA06             10 to 20        0.0        0.0        0.0  15 to 25   
NCA06             20 to 30        0.0        0.0        0.0   5 to 15   
NCA06     75dBA or greater        0.0        0.0        0.0      > 25   
NCA06                 > 30        0.0        0.0        0.0             
NCA07              0 to 10        0.0        0.0        0.0    0 to 5   
NCA07             10 to 20        1.0        0.0        0.0  15 to 25   
NCA07             20 to 30        0.0        0.0        1.0   5 to 15   
NCA07     75dBA or greater        1.0        0.0        1.0      > 25   
NCA07                 > 30        0.0        0.0        0.0             
OSR_COM            0 to 10        0.0        2.0        0.0    0 to 5   
OSR_COM           10 to 20        2.0        0.0   

In [ ]:
addishMitigayshBounds = {
    "MS": [0, 10, 20, 30],
    "Day": [0, 5, 10, 20],
    "OOH": [0, 5, 15, 25],
    "Eve": [0, 5, 15, 25],
    "ES": [0, 5, 15, 25],
    "Night": [0, 5, 15, 25],
    "SS": [0],
    "SA": [0]
}

addishMitigaysh = {
    "MS": ["boobs", "boobs", "boobs", "boobies"],
    "Day": ["boobs", "boobs", "boobs", "boobies"],
    "OOH": ["boobies", "boobies", "boobies", "boobies"],
    "Eve": ["A", "B", "C", "D"],
    "ES": ["E", "F", "G", "H"],
    "Night": ["I", "J", "K", "L"],
    "SS": ["AHH"],
    "SA": ["boobs"]
}

def makeMitigayshTable(resultsDf, mitigationBoundsDict, mitigationsDict, resultsColumns):
    mitigayshDf = resultsDf.copy()
    for col in resultsColumns:
        period = col.split('_')[1]
        nmlCol = f"{period}_NML"
        mitigayshDf[col] = resultsDf[col] - resultsDf[nmlCol]
        bounds = mitigationBoundsDict[period]
        measures = mitigationsDict[period]
        mitigayshDf[col] = np.searchsorted(bounds, mitigayshDf[col])
        mitigayshDf[col] = mitigayshDf[col].apply(lambda x: measures[x - 1] if x > 0 else "")
    return mitigayshDf


mitigayshOutDf = makeMitigayshTable(adjusted, addishMitigayshBounds, addishMitigaysh, filteredColumns)

print(mitigayshOutDf)



                          NCA_tool                     NCA_ID  \
NCA_ID                                                          
RES-NCA06-00190002497        NCA06      RES-NCA06-00190002497   
OSR_MXU-NCA06-00190007454    NCA06  OSR_MXU-NCA06-00190007454   
OSR_MXU-NCA07-00190007465    NCA07  OSR_MXU-NCA07-00190007465   
OSR_MXU-NCA07-00190007424    NCA07  OSR_MXU-NCA07-00190007424   
OSR_COM-NCA07-00190002328  OSR_COM  OSR_COM-NCA07-00190002328   
OSR_COM-NCA07-00190002373  OSR_COM  OSR_COM-NCA07-00190002373   
OSR_COM-NCA07-00190002372  OSR_COM  OSR_COM-NCA07-00190002372   
OSR_COM-NCA07-00190002346  OSR_COM  OSR_COM-NCA07-00190002346   
OSR_COM-NCA06-00190007403  OSR_COM  OSR_COM-NCA06-00190007403   
OSR_COM-NCA06-00190007395  OSR_COM  OSR_COM-NCA06-00190007395   
OSR_COM-NCA06-00190007393  OSR_COM  OSR_COM-NCA06-00190007393   
OSR_COM-NCA07-S372000018   OSR_COM   OSR_COM-NCA07-S372000018   
OSR_COM-NCA07-00190002349  OSR_COM  OSR_COM-NCA07-00190002349   
OSR_COM-NCA07-00280002222

In [83]:
## count tables for reports residential and osr

def makeResLeqLmaxCounts(stackedCount):
    stackedCount["Activity"] = stackedCount["Scenario"].str.split("_").str[0]
    rCDf = stackedCount[stackedCount["Receiver"].str.startswith("NCA")].copy()
    oCDf = stackedCount[stackedCount["Receiver"].str.startswith("OSR")].copy()

    activePeriods = list(set(p for p in stackedCount["Period"].values))
    sleepPeriods = ["SS", "SA"]
    leqPeriods = [p for p in activePeriods if p not in sleepPeriods]
    lmaxPeriods = [p for p in activePeriods if p in sleepPeriods]

    # residential pivot table (all periods lew and lmax)

    rCPt = pd.pivot_table(rCDf,
                        values="Count",
                        index=["Activity"],
                        columns=["Period", "Bin"],
                        aggfunc="sum").reset_index()

    # osr count tables to dictionary keyed by period
    osrPeriodCount = {}
    for p in leqPeriods:
        osrPeriodCount[f"{p}_OsrCount"] = pd.pivot_table(oCDf,
                                                        values="Count",
                                                        index="Activity",
                                                        columns=["Receiver", "Bin"],
                                                        aggfunc="sum").reset_index() 

    if leqPeriods:
        rCPtLeq = rCPt[["Activity"] + leqPeriods]

    if lmaxPeriods:
        rCPtSleep = rCPt[["Activity"] + lmaxPeriods]

    return rCPtLeq, rCPtSleep, osrPeriodCount

countResLeq, countResSleep, osrPeriodCount = makeResLeqLmaxCounts(stacked)


## return outputs only if they exist

print(countResLeq)
print(countResSleep)
print(osrPeriodCount)



Period Activity     Day                                             ES  \
Bin             0 to 10 10 to 20 20 to 30 75dBA or greater > 30 0 to 5   
0         SCEN1     0.0      1.0      0.0              1.0  0.0    0.0   
1         SCEN2     0.0      0.0      0.0              0.0  0.0    0.0   
2         SCEN3     0.0      0.0      1.0              1.0  0.0    0.0   

Period                          OOH                        Night           \
Bin    15 to 25 5 to 15 > 25 0 to 5 15 to 25 5 to 15 > 25 0 to 5 15 to 25   
0           0.0     0.0  0.0    0.0      1.0     0.0  0.0    0.0      0.0   
1           0.0     0.0  0.0    1.0      0.0     0.0  0.0    0.0      0.0   
2           0.0     0.0  0.0    0.0      0.0     0.0  1.0    0.0      0.0   

Period               
Bin    5 to 15 > 25  
0          0.0  0.0  
1          0.0  0.0  
2          0.0  0.0  
Period Activity   SA   SS
Bin              > 0  > 0
0         SCEN1  0.0  0.0
1         SCEN2  0.0  0.0
2         SCEN3  0.0  0.0
{'D

In [ ]:

## to do 
## move all calcs and outputs into this cell
## move all inputs e.g. dictionaries and variables into top cells
## then write all dfs to sheets in excel workbook 
## get input dfs from excel
## potentially look at formatting excel sheets